# Human activity prediction by modement
## 
Importing Libraries

In [8]:
import pandas as pd
import numpy as np

In [9]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [118]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'
DATADIR='C:/Users/suman/Downloads/appliedaidataset/UCI_HAR_Dataset'
import os
os.chdir(DATADIR)
#a=pd.read_csv(os.path.join('train','Inertial Signals','body_acc_x_train.txt')) 
a=pd.read_csv("train/Inertial Signals/total_acc_y_train.txt",delim_whitespace=True, header=None)
a.as_matrix()

#C:\Users\suman\Downloads\appliedaidataset\UCI_HAR_Dataset\train\Inertial Signals\body_acc_x_train

C:\Users\suman\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


array([[-0.1232167, -0.1268756, -0.1240037, ..., -0.1240696, -0.1227451,
        -0.121326 ],
       [-0.123976 , -0.1268078, -0.1277862, ..., -0.1290268, -0.1264244,
        -0.1305193],
       [-0.1200157, -0.1214994, -0.1228407, ..., -0.1308757, -0.1312099,
        -0.1301826],
       ...,
       [-0.1711829, -0.1999375, -0.2319457, ..., -0.2217656, -0.2811496,
        -0.3275988],
       [-0.223801 , -0.2210407, -0.1839923, ..., -0.1620087, -0.2020367,
        -0.2291273],
       [-0.3060512, -0.2666046, -0.2053607, ..., -0.3977751, -0.3492473,
        -0.3323721]])

In [86]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [160]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        #filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        #all the files has 7352 rows × 128 columns
        filename = subset+'/Inertial Signals/'+signal+'_'+subset+'.txt' 
        print('filename',filename)
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 
    #print(signals_data)    
    print('signals_data_shape',len(signals_data),len(signals_data[0]),len(signals_data[0][0]))
    #here its signals_data_shape 9 7352 128
    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    #now its changed to 7352 128 9
    return np.transpose(signals_data, (1, 2, 0))

In [161]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    #filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    filename = subset+'/y_'+subset+'.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [162]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [163]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [164]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [165]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [166]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [175]:
# Initializing parameters
epochs = 3
batch_size = 256
n_hidden = 16

In [168]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [169]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

filename train/Inertial Signals/body_acc_x_train.txt


C:\Users\suman\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app


filename train/Inertial Signals/body_acc_y_train.txt
filename train/Inertial Signals/body_acc_z_train.txt
filename train/Inertial Signals/body_gyro_x_train.txt
filename train/Inertial Signals/body_gyro_y_train.txt
filename train/Inertial Signals/body_gyro_z_train.txt
filename train/Inertial Signals/total_acc_x_train.txt
filename train/Inertial Signals/total_acc_y_train.txt
filename train/Inertial Signals/total_acc_z_train.txt
signals_data_shape 9 7352 128
filename test/Inertial Signals/body_acc_x_test.txt
filename test/Inertial Signals/body_acc_y_test.txt
filename test/Inertial Signals/body_acc_z_test.txt
filename test/Inertial Signals/body_gyro_x_test.txt
filename test/Inertial Signals/body_gyro_y_test.txt
filename test/Inertial Signals/body_gyro_z_test.txt
filename test/Inertial Signals/total_acc_x_test.txt
filename test/Inertial Signals/total_acc_y_test.txt
filename test/Inertial Signals/total_acc_z_test.txt
signals_data_shape 9 2947 128


C:\Users\suman\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


In [170]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))
print(X_train.shape)
print(Y_train.shape)

128
9
7352
(7352, 128, 9)
(7352, 6)


- Defining the Architecture of LSTM

In [91]:
# Initiliazing the sequential model
# try with different LSTM unit, dropout and more than 1 LSTM layer
model = Sequential()
# Configuring the parameters : model.add(keras.layers.LSTM(hidden_nodes, input_shape=(window, num_features)
#for sentence timesteps=no of words, input_dim=vector representation of a word
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
#here no of param: 4n(n+m+1) n=lstm cell m=input_dim   =9 : 4*32(32+9+1)=5376
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [23]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 92s 13ms/step - loss: 1.3018 - acc: 0.4395 - val_loss: 1.1254 - val_acc: 0.4662
Epoch 2/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.9666 - acc: 0.5880 - val_loss: 0.9491 - val_acc: 0.5714
Epoch 3/30
7352/7352 [==============================] - 97s 13ms/step - loss: 0.7812 - acc: 0.6408 - val_loss: 0.8286 - val_acc: 0.5850
Epoch 4/30
7352/7352 [==============================] - 95s 13ms/step - loss: 0.6941 - acc: 0.6574 - val_loss: 0.7297 - val_acc: 0.6128
Epoch 5/30
7352/7352 [==============================] - 92s 13ms/step - loss: 0.6336 - acc: 0.6912 - val_loss: 0.7359 - val_acc: 0.6787
Epoch 6/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.5859 - acc: 0.7134 - val_loss: 0.7015 - val_acc: 0.6939
Epoch 7/30
7352/7352 [==============================] - 95s 13ms/step - loss: 0.5692 - acc: 0.7477 - val_loss: 0.5995 - val_acc: 0.7387


In [24]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 512        0        25        0                   0   
SITTING                  3      410        75        0                   0   
STANDING                 0       87       445        0                   0   
WALKING                  0        0         0      481                   2   
WALKING_DOWNSTAIRS       0        0         0        0                 382   
WALKING_UPSTAIRS         0        0         0        2                  18   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            3  
STANDING                           0  
WALKING                           13  
WALKING_DOWNSTAIRS                38  
WALKING_UPSTAIRS                 451  


In [27]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 4s 2ms/step


In [28]:
aa=pd.DataFrame({'type':'LSTM 1 layer','dropout':'.5','score_loss':score[0],'score_accuracy':score[1]})                 

[0.3087582236972612, 0.9097387173396675]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

# Try with different dropout rate

In [176]:
# Initiliazing the sequential model
# try with different LSTM unit, dropout and more than 1 LSTM layer
model = Sequential()
# Configuring the parameters : model.add(keras.layers.LSTM(hidden_nodes, input_shape=(window, num_features)
#for sentence timesteps=no of words, input_dim=vector representation of a word
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
#here no of param: 4n(n+m+1) n=lstm cell m=input_dim   =9 : 4*32(32+9+1)=5376
# Adding a dropout layer
model.add(Dropout(0.2))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))


score = model.evaluate(X_test, Y_test)

bb=pd.DataFrame({'type':['LSTM 1 layer'],'dropout':['.2'],'score_loss':[score[0]],'score_accuracy':[score[1]]})                 
aa=aa.append(bb)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 16)                1664      
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 102       
Total params: 1,766
Trainable params: 1,766
Non-trainable params: 0
_________________________________________________________________
Train on 7352 samples, validate on 2947 samples
Epoch 1/3
7352/7352 [==============================] - 10s 1ms/step - loss: 1.7618 - acc: 0.2510 - val_loss: 1.7267 - val_acc: 0.4225
Epoch 2/3
7352/7352 [==============================] - 8s 1ms/step - loss: 1.6600 - acc: 0.4551 - val_loss: 1.6039 - val_acc: 0.4503
Epoch 3/3
7352/7352 [==============================] - 8s 1ms/step - loss: 1.5286 - acc: 

ValueError: If using all scalar values, you must pass an index

NameError: name 'aa' is not defined

# Try 2 layer of LSTM

In [ ]:
# Initiliazing the sequential model
# try with different LSTM unit, dropout and more than 1 LSTM layer
model = Sequential()
# Configuring the parameters : model.add(keras.layers.LSTM(hidden_nodes, input_shape=(window, num_features)
#for sentence timesteps=no of words, input_dim=vector representation of a word
model.add(LSTM(n_hidden, return_sequences=True,input_shape=(timesteps, input_dim)))
model.add(LSTM(n_hidden)) #,input_shape=(timesteps, input_dim))) #, stateful=True))

#here no of param: 4n(n+m+1) n=lstm cell m=input_dim   =9 : 4*32(32+9+1)=5376
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))


score = model.evaluate(X_test, Y_test)

bb=pd.DataFrame({'type':['LSTM 2 layer'],'dropout':['.5'],'score_loss':[score[0]],'score_accuracy':[score[1]]})                 
aa=aa.append(bb)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 128, 16)           1664      
_________________________________________________________________
lstm_21 (LSTM)               (None, 16)                2112      
_________________________________________________________________
dropout_4 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 102       
Total params: 3,878
Trainable params: 3,878
Non-trainable params: 0
_________________________________________________________________
Train on 7352 samples, validate on 2947 samples
Epoch 1/3
7352/7352 [==============================] - 20s 3ms/step - loss: 1.7021 - acc: 0.3293 - val_loss: 1.6196 - val_acc: 0.4136
Epoch 2/3
1792/7352 [======>.......................] - ETA: 12s - loss: 1.5971 - acc: 0.38